In [1]:
import numpy
import numpy as np
from numpy import asarray
from numpy import zeros
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential,Model
from keras.layers import *
from keras.layers import Dense, Flatten, Embedding
from keras.layers import LSTM, Dropout, Activation , Bidirectional,Dense,Flatten,Activation,RepeatVector,Permute,Multiply
from keras.layers.embeddings import Embedding
from keras.layers.core import*
from keras import initializers, regularizers, constraints, Input
from keras.layers.recurrent import LSTM
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras import backend as K
from keras.models import model_from_json
import codecs
import csv
from nltk import word_tokenize
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import emoji
import gensim
import time
import os
import sys 
import json
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

seed = 7
numpy.random.seed(seed)

Using TensorFlow backend.
W0427 19:26:45.376163 20360 transport.py:43] unable to import 'smart_open.gcs', disabling that module


In [2]:
def ReadOpen(filename,Labelfile):
    data = []
    with codecs.open(filename, 'r',encoding="utf-8", errors="replace") as readFile:
        reader = csv.reader(readFile)
        lines = list(reader)
    count = 0
    for i in lines:
        temp = []
        sentence = ' '.join(i)
        for j in word_tokenize(sentence):
            temp.append(j.lower()) 
            count += 1	  
        data.append(temp)
    labels_pd = pd.read_csv(Labelfile,index_col=False)
    labels = numpy.asarray(labels_pd)

    return data[1:],labels, count-1


def extract_emojis(sentence):
    return [word for word in sentence.split() if str(word.encode('unicode-escape'))[2] == '\\' ]


def char_is_emoji(character):
    if character in emoji.UNICODE_EMOJI:
        return True
    else:
        return False

In [3]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [4]:
def Preprocess(docs,count):
    # prepare tokenizer
    t = Tokenizer()
    t.fit_on_texts(docs)
    # vocab_size = len(t.word_index) + 1
    # print(vocab_size)
    # integer encode the documents
    encoded_docs = t.texts_to_sequences(docs)
    # pad documents to a max length of 4 words
    # max_length = 4
    padded_docs = pad_sequences(encoded_docs, padding='post')
    l = len(padded_docs[0])
    # load the whole embedding into memory
    embeddings_index = dict()
    f = open('glove.6B.300d.txt',encoding="utf8")
    for line in tqdm(f):
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))

    e2v = gensim.models.KeyedVectors.load_word2vec_format('emoji2vec.bin', binary=True)
    nf = 0
    # create a weight matrix for words in training docs
    embedding_matrix = zeros((count, 300))
    for word, i in t.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:
            # print(word)
            new_em = []
            em = extract_emojis(word)
            for ej in em:
                for c in ej:
                    if char_is_emoji(c):
                        new_em.append(c)
            # print(new_em)
            try:
                if new_em:
                        row = []
                        for e in new_em:

                            row.append(e2v[e])
                        embedding_matrix[i] = np.average(np.asarray(row),axis=0).tolist()
                else:
                    embedding_matrix[i] = [0] * 300
            except:
                embedding_matrix[i] = [0] * 300
                nf += 1

    print(str(nf)+" words not found in vocabulary")

    return padded_docs, embedding_matrix,l

In [10]:
def PrepModel(count,embedding_matrix,l,lrate=0.01):
    model = Sequential()
    e = Embedding(count, 300, weights=[embedding_matrix], input_length=l, trainable=False)
    model.add(e)
    model.add(Bidirectional(LSTM(100,kernel_initializer='he_normal', activation='sigmoid', dropout=0.5,recurrent_dropout=0.5, unroll=False, return_sequences=False)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(lr=lrate), loss='binary_crossentropy', metrics=['acc',f1,recall,precision])
    print('No of parameter:', model.count_params())
    print(model.summary())
    print(K.eval(model.optimizer.lr))
    return model


In [12]:
def ReadTest(filename,Labelfile):
    data = []

    with codecs.open(filename, 'r',encoding="utf-8", errors="replace") as readFile:
        reader = csv.reader(readFile)
        lines = list(reader)
    count = 0
    for i in lines:
        temp = []
        sentence = ' '.join(i)
        for j in word_tokenize(sentence):
            temp.append(j.lower()) 
            count += 1

        data.append(temp)

    labels_pd = pd.read_csv(Labelfile,index_col=False)
    labels = numpy.array(labels_pd['Labels'])
    # labels = numpy.asarray(labels_pd)

    t = Tokenizer()
    t.fit_on_texts(data)
    encoded_docs = t.texts_to_sequences(data)
    padded_docs = pad_sequences(encoded_docs, padding='post',maxlen=926)

    return padded_docs,labels



if __name__ == "__main__":

    with open('settings.json') as data_file:
        data = json.load(data_file)

    lrate = data["Model_settings"]["Learning_rate"]
    num_epochs = data["Model_settings"]["Epochs"]

    filename = data["FileNames"]["Training_file"]
    Labelfile = data["FileNames"]["Label_file"]
    # data,labels = ReadFile(filename,Labelfile)


    print('Reading data...')
    data,labels,count = ReadOpen(filename,Labelfile)
    print('Getting Embeddings...')
    padded_docs, embedding_matrix,l = Preprocess(data,count)
    print('Preparing model...')
    model = PrepModel(count,embedding_matrix,l,lrate)
    print('Training...')

    X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels, test_size=0.2, random_state=seed)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)
    # X_test1, y_test1 = ReadTest("Testing_data.csv","Testing_labels.csv")
    # print("length of X_test,y_test")
    # print(len(X_test),len(y_test))


    earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, verbose=1, mode='auto')
    model.fit(X_train, y_train, validation_data=(X_val,y_val), nb_epoch=3, verbose=1, callbacks=[earlyStopping])
    loss, accuracy,f1_score,recall_score,precision_score = model.evaluate(X_test, y_test, verbose=1)
    
    print('Accuracy: %f' % (accuracy*100))
    print('F1 Score: %f' % (f1_score*100))
    print('Recall: %f' % (recall_score*100))
    print('Precision: %f' % (precision_score*100))

Reading data...
Getting Embeddings...


400000it [00:43, 9152.52it/s]


Loaded 400000 word vectors.
235 words not found in vocabulary
Preparing model...
No of parameter: 29679901
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 926, 300)          29358900  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               320800    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 29,679,901
Trainable params: 321,001
Non-trainable params: 29,358,900
_________________________________________________________________
None
0.01
Training...


C:\Users\SIRAJAHAMED\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 3500 samples, validate on 876 samples
Epoch 1/3
2880/3500 [=======================>......] - ETA: 15:58 - loss: 0.7603 - acc: 0.4688 - f1: 0.6383 - recall: 1.0000 - precision: 0.46 - ETA: 10:42 - loss: 1.0318 - acc: 0.4531 - f1: 0.3191 - recall: 0.5000 - precision: 0.23 - ETA: 8:49 - loss: 0.9363 - acc: 0.4479 - f1: 0.2128 - recall: 0.3333 - precision: 0.1562 - ETA: 7:52 - loss: 0.8469 - acc: 0.5156 - f1: 0.3687 - recall: 0.5000 - precision: 0.296 - ETA: 7:16 - loss: 0.8966 - acc: 0.5125 - f1: 0.4283 - recall: 0.6000 - precision: 0.337 - ETA: 6:52 - loss: 0.9274 - acc: 0.5052 - f1: 0.4633 - recall: 0.6667 - precision: 0.359 - ETA: 6:35 - loss: 0.9429 - acc: 0.4955 - f1: 0.4840 - recall: 0.7143 - precision: 0.370 - ETA: 6:20 - loss: 0.9297 - acc: 0.4844 - f1: 0.4933 - recall: 0.7500 - precision: 0.372 - ETA: 6:09 - loss: 0.9044 - acc: 0.5000 - f1: 0.5218 - recall: 0.7576 - precision: 0.408 - ETA: 5:59 - loss: 0.8831 - acc: 0.5031 - f1: 0.5179 - recall: 0.7256 - precision: 0.421

2912/3500 [=======================>......] - ETA: 5:37 - loss: 0.4991 - acc: 0.7500 - f1: 0.7778 - recall: 0.8750 - precision: 0.700 - ETA: 5:29 - loss: 0.4974 - acc: 0.7656 - f1: 0.8111 - recall: 0.9375 - precision: 0.715 - ETA: 5:23 - loss: 0.5086 - acc: 0.7292 - f1: 0.7801 - recall: 0.9167 - precision: 0.679 - ETA: 5:19 - loss: 0.5249 - acc: 0.7109 - f1: 0.7680 - recall: 0.9081 - precision: 0.666 - ETA: 5:14 - loss: 0.5447 - acc: 0.6937 - f1: 0.7512 - recall: 0.8998 - precision: 0.645 - ETA: 5:12 - loss: 0.5461 - acc: 0.6823 - f1: 0.7371 - recall: 0.8832 - precision: 0.633 - ETA: 5:08 - loss: 0.5290 - acc: 0.6964 - f1: 0.7490 - recall: 0.8999 - precision: 0.642 - ETA: 5:04 - loss: 0.5141 - acc: 0.7148 - f1: 0.7589 - recall: 0.8756 - precision: 0.687 - ETA: 5:01 - loss: 0.5409 - acc: 0.6944 - f1: 0.7190 - recall: 0.8154 - precision: 0.666 - ETA: 4:58 - loss: 0.5593 - acc: 0.6813 - f1: 0.6932 - recall: 0.7713 - precision: 0.659 - ETA: 4:55 - loss: 0.5651 - acc: 0.6761 - f1: 0.6870 - r

3500/3500 [==============================] - ETA: 54s - loss: 0.5121 - acc: 0.7368 - f1: 0.7276 - recall: 0.7619 - precision: 0.72 - ETA: 51s - loss: 0.5105 - acc: 0.7379 - f1: 0.7290 - recall: 0.7632 - precision: 0.72 - ETA: 48s - loss: 0.5101 - acc: 0.7380 - f1: 0.7295 - recall: 0.7639 - precision: 0.72 - ETA: 45s - loss: 0.5095 - acc: 0.7391 - f1: 0.7310 - recall: 0.7658 - precision: 0.72 - ETA: 42s - loss: 0.5080 - acc: 0.7409 - f1: 0.7328 - recall: 0.7669 - precision: 0.73 - ETA: 38s - loss: 0.5092 - acc: 0.7407 - f1: 0.7329 - recall: 0.7679 - precision: 0.72 - ETA: 35s - loss: 0.5088 - acc: 0.7411 - f1: 0.7338 - recall: 0.7692 - precision: 0.73 - ETA: 32s - loss: 0.5088 - acc: 0.7412 - f1: 0.7341 - recall: 0.7696 - precision: 0.73 - ETA: 29s - loss: 0.5091 - acc: 0.7412 - f1: 0.7344 - recall: 0.7719 - precision: 0.72 - ETA: 26s - loss: 0.5094 - acc: 0.7416 - f1: 0.7346 - recall: 0.7726 - precision: 0.72 - ETA: 23s - loss: 0.5085 - acc: 0.7420 - f1: 0.7345 - recall: 0.7714 - preci